In [37]:

import qiskit
import numpy as np
import tqix
import sys
sys.path.insert(1, '../')
import qtm.base
import qtm.encoding

def create_basic_vector(num_qubits: int):
    """Generate list of basic vectors

    Args:
        num_qubits (int): number of qubits

    Returns:
        np.ndarray: |00...0>, |00...1>, ..., |11...1>
    """
    bs = []
    for i in range(0, 2**num_qubits):
        b = np.zeros((2**num_qubits, 1))
        b[i] = 1
        bs.append(b)
    return bs


def calculate_sigma(U: np.ndarray, b: np.ndarray):
    """Calculate measurement values

    Args:
        U (np.ndarray): operator
        b (np.ndarray): basic vector

    Returns:
        np.ndarray: sigma operator
    """
    return (np.conjugate(np.transpose(U)) @ b @ np.conjugate(np.transpose(b)) @ U)


Step 1. Create $\rho_{unk}$. It's psi


In [38]:
num_qubits = 2
num_observers = 10
psi = 2*np.random.rand(2**num_qubits)-1
psi = psi / np.linalg.norm(psi)
rho = qiskit.quantum_info.DensityMatrix(psi).data
qc = qiskit.QuantumCircuit(num_qubits, num_qubits)
qc.initialize(psi)


Step 2. Create Us = $U_1, U_2, ..., U_{num\_observer}$ and bs = $|0\rangle, |1\rangle, ..., |2^n-1\rangle$.

bs[0] = [1, 0, ... 0], bs[$2^n-1$] = [0, 0, ..., 1]


In [39]:
Us, bs = [], []
for i in range(0, num_observers):
    random_psi = 2*np.random.rand(2**num_qubits)-1
    random_psi = random_psi / np.linalg.norm(random_psi)
    encoder = qtm.encoding.Encoding(random_psi, 'amplitude_encoding')
    qcs = (qc.copy()).compose(encoder.qcircuit)
    U = (qiskit.quantum_info.Operator(encoder.qcircuit).data)
    Us.append(U)
    bs = create_basic_vector(num_qubits)


Step 3. Calculate $\sigma_i=\sigma_i^{(0)}, \sigma_i^{(1)}, ..., \sigma_i^{(2^n-1)}$
with $\sigma_i^{(j)}=U_i^{\dagger}|j\rangle\langle j|U_i$


In [40]:
sigmass = []
for i in range(0, num_observers):
    sigmas = []
    for b in bs:
        sigma = calculate_sigma(Us[i], b)
        sigmas.append(sigma)
    sigmass.append(sigmas)

Step 4: Calculate $\mu(\rho_{unk})=\frac{1}{num\_ observer}\sum_{i=1}^{num\_observer}\sum_{b=0}^{2^n-1} \text{Tr}(\sigma_i^{(b)}\rho_{unk})\sigma_i^{(b)}$


In [42]:
def calculate_mu(density_matrix):
    M = np.zeros((2**num_qubits, 2**num_qubits), dtype=np.complex128)
    for i in range(0, num_observers):
        for j in range(0, 2**num_qubits):
            k = sigmass[i][j]
            M += np.trace(k @ density_matrix) * k
    M /= num_observers
    return M
    
# M = calculate_mu(rho)

Step 5: Calculate $\tilde{\rho}=\frac{1}{num\_ observer}\sum_{i=1}^{num\_observer}(\sum_{b=0}^{2^n-1} (\text{Tr}(\sigma_i^{(b)}\rho_{unk}).\mu^{-1}(\sigma_i^{(b)})))$


In [ ]:
rho_hat = np.zeros((2**num_qubits, 2**num_qubits), dtype=np.complex128)
for i in range(0, num_observers):
    if i % 10 == 0:
        print(i)
    for j in range(0, 2**num_qubits):
        k = sigmass[i][j]
        rho_hat += np.trace(k @ rho)*np.linalg.inv(calculate_mu(k))
rho_hat /= num_observers
new_rho_hat = (np.conjugate(np.transpose(
    rho_hat)) @ rho_hat) / (np.trace(np.conjugate(np.transpose(rho_hat)) @ rho_hat))


Step 5.1: Calculate $\tilde{\rho}=\sum_{i=1}^{num\_observer}p_i(\sum_{b=0}^{2^n-1} (\text{Tr}(\sigma_i^{(b)}\rho_{unk}).\mu^{-1}(\sigma_i^{(b)})))$

<img width = '600px' src = '../../images/shadow_tomography3.jpg'>

In [46]:
from scipy.stats import truncnorm

def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
X = get_truncated_normal(mean= num_observers/2, sd=2, low=0, upp = num_observers).rvs(100)

# Calculate p_i
probs = np.zeros(10)
for i in range(0, 100):
    probs[int(X[i])] += 1
probs /= np.sum(probs)


rho_hat = np.zeros((2**num_qubits, 2**num_qubits), dtype=np.complex128)
for i in range(0, num_observers):
    matrix_tmp = np.zeros((2**num_qubits, 2**num_qubits), dtype=np.complex128)
    for j in range(0, 2**num_qubits):
        k = sigmass[i][j]
        matrix_tmp += np.trace(k @ rho)*np.linalg.inv(calculate_mu(k))
    rho_hat += probs[i]*matrix_tmp


new_rho_hat = (np.conjugate(np.transpose(
    rho_hat)) @ rho_hat) / (np.trace(np.conjugate(np.transpose(rho_hat)) @ rho_hat))


In [47]:
print("p", rho)
print("p~", new_rho_hat)
fidelity = qtm.base.trace_fidelity(rho, new_rho_hat)
trace = qtm.base.trace_distance(rho, new_rho_hat)
print("Fidelity: ", fidelity)
print("Trace: ", trace)


p [[ 0.03761789+0.j -0.13110046+0.j  0.00928232+0.j -0.13758378+0.j]
 [-0.13110046-0.j  0.45689242+0.j -0.03234942-0.j  0.47948714+0.j]
 [ 0.00928232+0.j -0.03234942+0.j  0.00229044+0.j -0.03394919+0.j]
 [-0.13758378-0.j  0.47948714+0.j -0.03394919-0.j  0.50319925+0.j]]
p~ [[ 0.44313979+0.j  0.11756903+0.j -0.34020174+0.j -0.06387691+0.j]
 [ 0.11756903+0.j  0.13893405+0.j -0.0848641 +0.j -0.06254983+0.j]
 [-0.34020174+0.j -0.0848641 +0.j  0.36634562+0.j  0.07796326+0.j]
 [-0.06387691+0.j -0.06254983+0.j  0.07796326+0.j  0.05158053+0.j]]
Fidelity:  (0.1360777123501563-5.082341657911845e-18j)
Trace:  0.9749525014506578


## Materials

<img width = '600px' src = '../../images/shadow_tomography1.jpg'>
<img width = '600px' src = '../../images/shadow_tomography2.jpg'>
